# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」における人間の好み予測タスクに取り組んでいます。主な目的は、ユーザーが異なるチャットボットからの応答の中でどちらを選ぶかを予測するためのモデルを開発することです。ノートブックではトレーニングデータのサイズが大きいため、そのわずか0.5%のみを使用して、データのクリーニングと特徴量エンジニアリングを行います。

### 使用される手法やライブラリ
1. **データ処理と分析**:
   - `pandas`, `numpy`: データフレームの操作や数値計算。
   - `textstat`: テキストの読みやすさや統計分析。
   - `SweetViz`: データ分析と視覚化。

2. **自然言語処理**:
   - `nltk`: ストップワードの取得やテキストの分析。
   - `transformers`: BERTモデルを用いたトークン化と埋め込み生成。

3. **機械学習**:
   - `scikit-learn`: データの分割、モデル評価、さまざまな分類器（ランダムフォレスト、勾配ブースティング、サポートベクターマシンなど）の実装。
   - `tensorflow`: ニューラルネットワークモデルの構築とトレーニング。

### 主な工程
- **データの読み込みと前処理**: トレーニングデータとテストデータをCSVから読み込み、クリーニングを実施（特殊文字の削除、正規化など）。
- **特徴量エンジニアリング**: テキストの単語数、文字数、感情分析、およびBERTを用いた埋め込み生成を含む多様な特徴量を抽出。
- **モデルの構築と評価**: 複数の機械学習モデルをトレーニングし、その性能を精度やログ損失を用いて評価。最良のモデルを選択します。
- **テストデータに対する予測**: 最良モデルを用いてテストデータに対する予測を実施し、提出用フォーマットでCSVファイルを生成。

このノートブックは、機械学習や自然言語処理を用いて実際の問題にアプローチし、チャットボットの応答のユーザー選好を予測するための包括的なフレームワークを提供しています。

---


# 用語概説 
以下は、Jupyter Notebookにおいて初心者がつまずきそうな専門用語の簡単な解説です。特にこのノートブック特有のドメイン知識や、実務未経験者にはなじみが薄い用語に焦点を当てています。

1. **TPU (Tensor Processing Unit)**:
   - Googleが設計した特化型ハードウェアで、特に機械学習のために最適化されているプロセッサです。通常のGPUよりも大規模なモデルを高速にトレーニングできます。

2. **埋め込み (Embedding)**:
   - 単語や文などの高次元のデータを低次元のベクトルに変換する手法です。意味情報を保持しつつ、計算効率を向上させるために使います。BERTのような言語モデルでは、言語の意味を理解するために重要です。

3. **VADER (Valence Aware Dictionary and sEntiment Reasoner)**:
   - 主にソーシャルメディアでの感情分析に対応した辞書およびルールベースの手法です。ポジティブやネガティブな感情をスコアとして評価し、特に短いテキストや感情的な表現に効果的です。

4. **Flesch Reading Easeスコア**:
   - テキストの読みやすさを数値化した指標で、高いスコアほど読みやすいことを示します。文の長さや音節数を考慮して計算されます。

5. **コサイン類似度 (Cosine Similarity)**:
   - 二つのベクトルのコサインの角度を基準にした類似度指標です。1に近いほど似ていることを示し、主に情報検索や自然言語処理で使用されます。

6. **ドロップアウト (Dropout)**:
   - ニューラルネットワークの訓練中に、過学習を防ぐためにランダムに一定の割合のニューロンを「ドロップ」する手法です。これにより、モデルの一般化能力が向上します。

7. **ログ損失 (Log Loss)**:
   - 分類問題におけるモデルの性能を評価するための指標で、確率的な予測に対する損失を計算します。モデルが予測した確率と実際のラベルとの違いを反映します。

8. **シーケンス (Sequence)**:
   - 一連のデータポイントの配列で、通常は時系列データやテキストのように順序が重要なデータに使用されます。

9. **トークン化 (Tokenization)**:
   - テキストを単語やフレーズといった「トークン」に分割する過程です。機械学習モデルがテキストを理解するために不可欠な第一歩です。

10. **特徴量エンジニアリング (Feature Engineering)**:
    - 生データから機械学習モデルが有用な情報を引き出すための特徴量を生成、選択、変換するプロセスです。モデル性能に大きく影響します。

この解説リストが、ノートブックを理解するのに役立つことを願っています。

---


# LMSYS - Chatbot Arenaでの人間の好み予測



**トレーニングデータのサイズが大きいため、私はトレーニングデータのわずか0.5%のみを使用しています！**

作業中：TPUを使用して別のノートブックで埋め込みを計算し、フルトレーニングデータを使用した後、ここに埋め込みをロードします！

## ライブラリのインストールと読み込み

In [ ]:
!pip install textstat SweetViz  # textstatとSweetVizをインストールします。

# textstatはテキストの統計情報を分析するためのライブラリで、例えば読みやすさのスコアを計算することができます。
# SweetVizはデータの可視化を提供するライブラリで、データセットの探索に役立ちます。

In [ ]:
import warnings  # 警告を表示しないように設定します。
warnings.filterwarnings('ignore')  # すべての警告を無視します。

import pandas as pd  # データ操作のためのライブラリ
import numpy as np  # 数値計算のためのライブラリ

import seaborn as sns  # データの可視化に役立つライブラリ
import matplotlib.pyplot as plt  # グラフを描画するためのライブラリ
import plotly.express as px  # インタラクティブなグラフを描画するためのライブラリ
import sweetviz as sv  # データセットの探索に役立つ可視化ライブラリ

import re  # 正規表現を扱うためのライブラリ
import string  # 文字列操作のためのライブラリ
import nltk  # 自然言語処理のためのライブラリ
from nltk.corpus import stopwords  # ストップワード（意味を持たない単語）のリストを取得するためのモジュール

import tensorflow as tf  # 機械学習と深層学習のフレームワーク
from tensorflow.keras.preprocessing.text import Tokenizer  # テキストのトークン化を行うためのクラス
from tensorflow.keras.preprocessing.sequence import pad_sequences  # シーケンスを一定の長さにパディングするための関数

from nltk.sentiment.vader import SentimentIntensityAnalyzer  # 感情分析を行うためのモジュール

import textstat  # テキストの統計情報を計算するためのライブラリ
from sklearn.feature_extraction.text import CountVectorizer  # テキストデータを数値に変換するためのクラス

from sklearn.model_selection import train_test_split  # データセットをトレーニングとテストに分割するための関数

from sklearn.metrics.pairwise import cosine_similarity  # コサイン類似度を計算するための関数
from transformers import BertTokenizer, TFBertModel  # BERTトークン化とBERTモデルを使用するためのライブラリ

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier  # ランダムフォレストと勾配ブースティングのクラス
from sklearn.svm import SVC  # サポートベクターマシンのクラス

from tensorflow.keras.models import Sequential  # Kerasのシーケンシャルモデル
from tensorflow.keras.layers import Dense, Dropout  # 全結合層とドロップアウト層
from tensorflow.keras.optimizers import Adam  # Adamオプティマイザ

from sklearn.metrics import accuracy_score, \  # 精度を評価するための関数
                            log_loss, \  # ログ損失を評価するための関数
                            confusion_matrix, \  # 混同行列を計算するための関数
                            classification_report  # 分類結果のレポートを生成するための関数

In [ ]:
nltk.download('stopwords')  # NLTKライブラリからストップワードのリストをダウンロードします。
stop_words = set(stopwords.words('english'))  # 英語のストップワードを取得し、セットとして格納します。

# ストップワードとは、テキストデータの中で特に意味を持たない単語のことであり、
# これを除外することで、テキスト分析の精度を向上させることができます。

## データの読み込み

In [ ]:
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータをCSVファイルから読み込みます。
test_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータをCSVファイルから読み込みます。

# pd.read_csv関数は、指定されたファイルパスからCSV形式のデータを読み込むための関数です。
# 読み込まれたデータはpandasのDataFrame形式で格納され、データの操作や分析に利用できます。

## 探索的データ分析（EDA）

In [ ]:
train_analysis = sv.analyze(train_data)  # トレーニングデータに対してSweetVizを用いて分析を行います。

# sv.analyze関数は、指定されたデータセットに関する詳細なレポートを生成し、
# データの特性や分布を視覚的に示すための情報を提供します。 
# このレポートは、データの概要を把握するのに役立ちます。

In [ ]:
train_analysis.show_html('train_analysis.html')  # SweetVizによるトレーニングデータの分析結果をHTMLファイルとして保存します。

# show_html関数は、分析結果をHTML形式で出力し、指定されたファイル名で保存します。
# このHTMLファイルは、データの可視化や分析結果をブラウザで確認するのに便利です。

In [ ]:
train_data.head()  # トレーニングデータの最初の5行を表示します。

# head()関数は、DataFrameの最初の数行を表示するための関数で、
# データの構造や内容をすばやく確認するのに役立ちます。
# 引数を指定することで、表示する行数を変更することも可能です。デフォルトでは5行表示されます。

In [ ]:
print("Training Data -", train_data.shape)  # トレーニングデータの形状（行数と列数）を表示します。
print("Test Data -", test_data.shape)  # テストデータの形状（行数と列数）を表示します。

# shape属性は、データフレームの行数と列数をタプル形式で返します。
# これにより、データセットのサイズを簡単に把握することができます。

In [ ]:
train_data.describe(include=['O'])  # トレーニングデータのオブジェクト型（文字列）の列に関する要約統計量を表示します。

# describe()関数は、データフレームの基本的な統計情報を提供します。
# include=['O']を指定することで、オブジェクト型の列（通常は文字列データ）のみを対象にした要約が取得できます。
# 出力には、ユニークな値の数や最頻値などが含まれ、データの特徴を把握するのに役立ちます。

In [ ]:
print(train_data.info())  # トレーニングデータの情報を表示します。

# info()関数は、データフレームの構造に関する詳細情報を提供します。
# 出力には、各列のデータ型、非NULL値のカウント、メモリの使用量などが含まれます。
# これにより、データセットの各列の状態を簡単に確認することができます。

In [ ]:
train_data.drop("id", axis=1).duplicated().sum()  # "id"列を除いたトレーニングデータの重複行の数をカウントします。

# drop("id", axis=1)は、"id"という列をデータフレームから削除します。
# duplicated()メソッドは、重複している行を検出し、重複行に対してTrueを返します。
# sum()は、Trueの合計を計算し、重複行の総数を返します。 
# これにより、データセット内の重複するデータの有無を確認できます。

14行の重複が存在し、7つのグループを形成しています。私は各グループごとに1行のみを保持します。

In [ ]:
train_data = train_data.drop_duplicates(keep="first", ignore_index=True)  # 重複行を削除し、先頭の行を保持します。

# drop_duplicates()メソッドは、データフレーム内の重複行を削除します。
# 引数keep="first"は、最初の出現を保持し、他の重複行を削除することを指定します。
# ignore_index=Trueを指定することで、インデックスをリセットし、新たな連続したインデックスを付与します。
# これにより、重複行が削除された後でも、インデックスが整然とした状態になります。

`id`列に関してトレーニングデータの品質を確認します。

In [ ]:
train_data.nunique()  # トレーニングデータの各列におけるユニークな値の数を表示します。

# nunique()メソッドは、データフレーム内の各列においてユニークな値の個数を計算します。
# この情報は、各列のデータにどれだけのバリエーションがあるかを把握するのに役立ちます。

In [ ]:
assert train_data["id"].nunique() == len(train_data)  # `id`列のユニークな値の数がトレーニングデータの行数と等しいことを確認します。

# assert文は、指定された条件がTrueであることをチェックします。
# もし条件が満たされない場合、エラーが発生し、プログラムの実行が停止します。
# この条件により、`id`列に重複がないこと（すなわち、全ての`id`がユニークであること）を確認しています。

In [ ]:
train_data.isna().sum()  # トレーニングデータの各列における欠損値（NaN）の合計を表示します。

# isna()メソッドは、データフレームの各セルが欠損値であるかどうかを確認し、ブーリアン値（TrueまたはFalse）を返します。
# sum()関数を使用することで、各列に含まれる欠損値の合計を計算し、結果を表示します。
# この情報は、データのクリーニングや前処理において欠損値の処理を行う前に確認するのに役立ちます。

### 分布

In [ ]:
model_df = pd.concat([train_data.model_a, train_data.model_b])  # model_aとmodel_bの列を結合します。
counts = model_df.value_counts().reset_index()  # 各モデルの出現回数をカウントし、データフレームにリセットします。
counts.columns = ['LLM', 'Count']  # カラム名を設定します。

# Plotlyを使用してカスタムスタイルの棒グラフを作成します。
fig = px.bar(counts, x='LLM', y='Count',
             title='モデルの分布',
             color='Count')  # モデルに応じた色を設定します。

fig.update_layout(xaxis_tickangle=-45)  # x軸のラベルを45度回転させて見やすくします。

fig.show()  # グラフを表示します。

In [ ]:
counts_a = train_data['winner_model_a'].value_counts().reset_index()  # モデルAの勝利数をカウントします。
counts_b = train_data['winner_model_b'].value_counts().reset_index()  # モデルBの勝利数をカウントします。
counts_tie = train_data['winner_tie'].value_counts().reset_index()  # 引き分けの数をカウントします。

# 分かりやすくするためにカラム名を変更します。
counts_a.columns = ['Winner', 'Count']
counts_b.columns = ['Winner', 'Count']
counts_tie.columns = ['Winner', 'Count']

# モデルを識別するためのカラムを追加します。
counts_a['Model'] = 'Model A'
counts_b['Model'] = 'Model B'
counts_tie['Model'] = 'Tie'

# すべてのカウントを結合します。
counts = pd.concat([counts_a, counts_b, counts_tie])

# カスタムスタイルの棒グラフを作成します。
fig = px.bar(counts, x='Model', y='Count', 
             color='Model',
             title='トレーニングデータの勝者分布',
             labels={'Model': 'モデル', 'Count': '勝利数', 'Winner': '勝者'})

fig.update_layout(xaxis_title="モデル", yaxis_title="勝利数")  # x軸とy軸のタイトルを設定します。

fig.show()  # グラフを表示します。

結論:

* トレーニングデータは57,477行、テストデータは3行です。
    * 注: スコアリングフェーズ中にテストデータは完全なテストセット（約25,000行、70%はプライベートLB用）に置き換えられます。
* `id`列には重複値がありません。
* モデルの識別子はテストセットでは明らかにされていません。
* `prompt`、`response_a`、`response_b`の各列の文字列はリストにラップされています。
    * これは、各チャットが複数のプロンプト/レスポンスのペアを含む可能性があるためです。
* `id`列を削除した後、14行の重複が存在し、7つのグループを形成しています。各グループごとに1行だけを保持した結果、トレーニングデータフレームの形状は(57,470, 8)になります。

## データ準備と特徴量エンジニアリング

* データのクリーニング: 特殊文字を削除し、小文字に正規化し、ストップワードを削除し、トークン化するなどのテキストをクリーンアップします。
* 入力のトークン化: トレーニングデータに基づいてTensorFlow/Kerasのトークナイザーを使用し、トレーニングデータとテストデータの両方にフィットさせます。
* シーケンスを`max_len`にパディングします。
* BERT埋め込みを作成します。
* 各モデルのプロンプトとレスポンス間でBERTを使用した類似度特徴量を計算します。
* 各レスポンスについて、単語数、文字数、語彙の多様性を計算します。
* BERTモデル用にテキスト入力をトークン化します。

In [ ]:
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv').sample(frac=0.001)  # トレーニングデータをCSVファイルから読み込み、全体の0.1%をサンプリングします。

# read_csv()関数は、指定されたファイルパスからCSV形式のデータを読み込みます。
# sample(frac=0.001)は、データフレームから全体の0.1%（0.001）のランダムなサンプルを抽出します。
# これにより、大規模なデータセットの一部を使用して迅速に実験や分析を行うことができます。

### データのクリーニング

特殊文字を削除し、小文字に正規化し、ストップワードを削除するなどのクリーンなテキストを作成します。

In [ ]:
def clean_text(text):  # テキストをクリーンアップするための関数を定義します。
    text = text.lower()  # テキストを小文字に変換します。
    text = re.sub(r'\[.*?\]', '', text)  # 括弧内の内容を削除します。
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # URLを削除します。
    text = re.sub(r'<.*?>+', '', text)  # HTMLタグを削除します。
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # 句読点を削除します。
    text = re.sub(r'\n', '', text)  # 改行を削除します。
    text = re.sub(r'\w*\d\w*', '', text)  # 数字を含む単語を削除します。
    text = ' '.join(word for word in text.split() if word not in stop_words)  # ストップワードを削除し、単語を再結合します。
    return text  # クリーンアップされたテキストを返します。

In [ ]:
# テキストのクリーニングを行います。
train_data['prompt_clean'] = train_data['prompt'].apply(clean_text)  # 'prompt'列にclean_text関数を適用し、クリーニングした結果を'prompt_clean'列に格納します。
train_data['response_a_clean'] = train_data['response_a'].apply(clean_text)  # 'response_a'列にclean_text関数を適用し、クリーニングした結果を'response_a_clean'列に格納します。
train_data['response_b_clean'] = train_data['response_b'].apply(clean_text)  # 'response_b'列にclean_text関数を適用し、クリーニングした結果を'response_b_clean'列に格納します。

# apply()メソッドは、指定した関数をデータフレームの各要素に適用し、新しいシリーズを返します。
# これにより、元のテキストデータがクリーニングされ、分析やモデルの入力に適した形式へと変換されます。

### 入力のトークン化

TensorFlow/Kerasのトークナイザーを使用して、トレーニングデータとテストデータの両方に対してトークン化を行います。シーケンスを`max_len`にパディングします。

In [ ]:
max_len = 512  # シーケンスの最大長を512に設定します。

# max_lenは、BERTなどのトランスフォーマーモデルで処理できる入力シーケンスの最大の長さを指定します。
# これにより、すべての入力が均一な長さになるようにパディングやカットが適用されます。

In [ ]:
tokenizer = Tokenizer(num_words=20000)  # 語彙サイズを20,000語に制限したトークナイザーを初期化します。

# Tokenizerクラスは、テキストを数値のトークンに変換するためのもので、num_words引数は使用する語彙の最大数を指定します。
# これにより、頻出の単語を保持し、あまり使われない単語は除外されるため、モデルの効率が向上します。

In [ ]:
tokenizer.fit_on_texts(pd.concat([train_data['prompt_clean'], train_data['response_a_clean'], train_data['response_b_clean']]))  # トークナイザーをトレーニングデータのクリーニングされたプロンプトとレスポンスでフィットさせます。

# fit_on_texts()メソッドは、指定されたテキストデータから単語の頻度に基づいて語彙を構築します。
# pd.concat()を使用して、プロンプトとレスポンスを結合し、一緒にトークン化することで、すべてのテキストに基づいた語彙を作成します。

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_data['prompt_clean'])  # クリーニングされたプロンプトを数値のシーケンスに変換します。
response_a_sequences = tokenizer.texts_to_sequences(train_data['response_a_clean'])  # クリーニングされたレスポンスAを数値のシーケンスに変換します。
response_b_sequences = tokenizer.texts_to_sequences(train_data['response_b_clean'])  # クリーニングされたレスポンスBを数値のシーケンスに変換します。

# texts_to_sequences()メソッドは、テキストを数値のシーケンスに変換し、各単語を対応するインデックスに置き換えます。
# これにより、モデルに入力できる形式のデータが得られ、後の処理や学習のために準備されます。

In [ ]:
# パディングを行います。
train_sequences = pad_sequences(train_sequences, maxlen=max_len, padding='post')  # プロンプトのシーケンスを最大長にパディングします。シーケンスの末尾にパディングを追加します。
response_a_sequences = pad_sequences(response_a_sequences, maxlen=max_len, padding='post')  # レスポンスAのシーケンスを最大長にパディングします。
response_b_sequences = pad_sequences(response_b_sequences, maxlen=max_len, padding='post')  # レスポンスBのシーケンスを最大長にパディングします。

# pad_sequences()は、すべてのシーケンスを指定した最大長に整形し、短いシーケンスにはゼロや指定したパディング値を追加します。
# padding='post'を指定することで、シーケンスの後ろにパディングを追加し、長さを統一します。これにより、モデルの入力データが整った形式になります。

### 感情分析

`vaderSentiment`を使用した感情分析。VADER（Valence Aware Dictionary and sEntiment Reasoner）は、特にソーシャルメディアで表現される感情に調整された、辞書およびルールベースの感情分析ツールです。

In [ ]:
analyzer = SentimentIntensityAnalyzer()  # 感情分析のためのSentimentIntensityAnalyzerを初期化します。

# SentimentIntensityAnalyzerは、テキストの感情を評価するための便利なツールで、
# 各テキストに対してポジティブ、ネガティブ、ニュートラルなスコアを生成し、
# 全体的な感情の強度を示すコンパウンドスコアも計算します。

In [ ]:
def sentiment_analysis(text):  # 感情分析を行う関数を定義します。
    return analyzer.polarity_scores(text)['compound']  # テキストの感情スコアを計算し、コンパウンドスコアを返します。

# polarity_scores()メソッドは、入力テキストに対して感情のポジティブ、ネガティブ、ニュートラル、およびコンパウンドスコアを計算します。
# コンパウンドスコアは、全体的な感情の強度を示し、-1（非常にネガティブ）から1（非常にポジティブ）の範囲で評価されます。

In [ ]:
train_data['sentiment_prompt'] = train_data['prompt_clean'].apply(sentiment_analysis)  # クリーニングされたプロンプトに対して感情分析を行い、結果を'sentiment_prompt'列に格納します。
train_data['sentiment_response_a'] = train_data['response_a_clean'].apply(sentiment_analysis)  # クリーニングされたレスポンスAに対して感情分析を行い、結果を'sentiment_response_a'列に格納します。
train_data['sentiment_response_b'] = train_data['response_b_clean'].apply(sentiment_analysis)  # クリーニングされたレスポンスBに対して感情分析を行い、結果を'sentiment_response_b'列に格納します。

# apply()メソッドを使用して、各クリーニングされたテキストにsentiment_analysis関数を適用することで、
# 各テキストの感情スコアを計算し、新しい列としてデータフレームに追加します。

### テキスト特徴量

単語数、文字数、語彙の多様性、音節数、文の数、各レスポンスの読みやすさを定量的に測定するFlesch Reading Easeスコアなどのテキスト特徴量を計算します。

テキスト統計を分析するために`textstat`ライブラリを使用しています。

In [ ]:
def word_count(text):  # 単語数を計算する関数を定義します。
    return len(text.split())  # テキストをスペースで分割し、単語数を返します。

def char_count(text):  # 文字数を計算する関数を定義します。
    return len(text)  # テキストの全体の長さ（文字数）を返します。

def lexical_diversity(text):  # 語彙の多様性を計算する関数を定義します。
    words = text.split()  # テキストを単語に分割します。
    return len(set(words)) / len(words) if words else 0  # ユニークな単語の数を全単語数で割り、語彙の多様性を計算します。

def syllable_count(text):  # 音節数を計算する関数を定義します。
    return textstat.syllable_count(text)  # textstatライブラリを使用して音節数を返します。

def sentence_count(text):  # 文の数を計算する関数を定義します。
    return textstat.sentence_count(text)  # textstatライブラリを使用して文の数を返します。

def flesch_reading_ease(text):  # Flesch Reading Easeスコアを計算する関数を定義します。
    return textstat.flesch_reading_ease(text)  # textstatライブラリを使用してFlesch Reading Easeスコアを返します。

In [ ]:
train_data['word_count_prompt'] = train_data['prompt_clean'].apply(word_count)  # クリーニングされたプロンプトに対して単語数を計算し、'word_count_prompt'列に格納します。
train_data['word_count_response_a'] = train_data['response_a_clean'].apply(word_count)  # クリーニングされたレスポンスAに対して単語数を計算し、'word_count_response_a'列に格納します。
train_data['word_count_response_b'] = train_data['response_b_clean'].apply(word_count)  # クリーニングされたレスポンスBに対して単語数を計算し、'word_count_response_b'列に格納します。
train_data['char_count_prompt'] = train_data['prompt_clean'].apply(char_count)  # クリーニングされたプロンプトに対して文字数を計算し、'char_count_prompt'列に格納します。
train_data['char_count_response_a'] = train_data['response_a_clean'].apply(char_count)  # クリーニングされたレスポンスAに対して文字数を計算し、'char_count_response_a'列に格納します。
train_data['char_count_response_b'] = train_data['response_b_clean'].apply(char_count)  # クリーニングされたレスポンスBに対して文字数を計算し、'char_count_response_b'列に格納します。
train_data['lexical_diversity_prompt'] = train_data['prompt_clean'].apply(lexical_diversity)  # クリーニングされたプロンプトの語彙の多様性を計算し、'lexical_diversity_prompt'列に格納します。
train_data['lexical_diversity_response_a'] = train_data['response_a_clean'].apply(lexical_diversity)  # クリーニングされたレスポンスAの語彙の多様性を計算し、'lexical_diversity_response_a'列に格納します。
train_data['lexical_diversity_response_b'] = train_data['response_b_clean'].apply(lexical_diversity)  # クリーニングされたレスポンスBの語彙の多様性を計算し、'lexical_diversity_response_b'列に格納します。
train_data['syllable_count_prompt'] = train_data['prompt_clean'].apply(syllable_count)  # クリーニングされたプロンプトの音節数を計算し、'syllable_count_prompt'列に格納します。
train_data['syllable_count_response_a'] = train_data['response_a_clean'].apply(syllable_count)  # クリーニングされたレスポンスAの音節数を計算し、'syllable_count_response_a'列に格納します。
train_data['syllable_count_response_b'] = train_data['response_b_clean'].apply(syllable_count)  # クリーニングされたレスポンスBの音節数を計算し、'syllable_count_response_b'列に格納します。
train_data['sentence_count_prompt'] = train_data['prompt_clean'].apply(sentence_count)  # クリーニングされたプロンプトの文の数を計算し、'sentence_count_prompt'列に格納します。
train_data['sentence_count_response_a'] = train_data['response_a_clean'].apply(sentence_count)  # クリーニングされたレスポンスAの文の数を計算し、'sentence_count_response_a'列に格納します。
train_data['sentence_count_response_b'] = train_data['response_b_clean'].apply(sentence_count)  # クリーニングされたレスポンスBの文の数を計算し、'sentence_count_response_b'列に格納します。
train_data['flesch_reading_ease_prompt'] = train_data['prompt_clean'].apply(flesch_reading_ease)  # クリーニングされたプロンプトのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_prompt'列に格納します。
train_data['flesch_reading_ease_response_a'] = train_data['response_a_clean'].apply(flesch_reading_ease)  # クリーニングされたレスポンスAのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_response_a'列に格納します。
train_data['flesch_reading_ease_response_b'] = train_data['response_b_clean'].apply(flesch_reading_ease)  # クリーニングされたレスポンスBのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_response_b'列に格納します。

### BERT埋め込みの作成

トレーニングデータおよびテストデータのプロンプトとレスポンスに対してBERT埋め込みを計算します。また、各モデルのプロンプトとレスポンス間でBERTを使用したコサイン類似度特徴量も計算します。

効率的なパイプラインを作成するために`tf.data.Dataset`を使用し、GPUを利用してバッチで特徴量を処理します。また、`joblib`ライブラリを使用して中間埋め込みを保存する予定です。

In [ ]:
# BERTモデルをロードします。
bert_model_name = 'bert-base-uncased'  # 使用するBERTモデルの名前を指定します。
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)  # BERTトークナイザーを事前学習済みモデルから初期化します。
bert_model = TFBertModel.from_pretrained(bert_model_name)  # BERTモデルを事前学習済みモデルから初期化します。

# BertTokenizerはテキストをBERTモデル用にトークン化するためのもので、
# TFBertModelはTensorFlowで動作するBERTモデル本体を提供します。
# これにより、自然言語処理タスクにおいてBERTを利用できるようになります。

In [ ]:
@tf.function  # TensorFlowの関数として最適化されることを示します。
def get_bert_embeddings(texts):  # テキストに対してBERT埋め込みを取得する関数を定義します。
    inputs = bert_tokenizer(texts,  # テキストをトークナイズします。
                       return_tensors='tf',  # TensorFlowテンソルとして返します。
                       padding=True,  # パディングを有効にします。
                       truncation=True,  # テキストがmax_lengthを超える場合は切り捨てます。
                       max_length=512)  # 最大入力長を512に設定します。
    outputs = bert_model(inputs)  # BERTモデルを使用して埋め込みを生成します。
    return outputs.last_hidden_state[:, 0, :]  # 最後の隠れ層の出力の最初のトークン（[CLS]トークン）の埋め込みを返します。

# この関数は、与えられたテキストからBERT埋め込みを効率的に計算し、
# 特に分類タスクなどで重要な役割を果たす[CLS]トークンの埋め込みを取得します。

In [ ]:
def process_column(column_data):  # 指定された列のデータを処理する関数を定義します。
    column_data = column_data.dropna().tolist()  # 欠損値を削除し、リストに変換します。
    column_data = [str(text) for text in column_data]  # 各テキストを文字列に変換します。  
    dataset = tf.data.Dataset.from_tensor_slices(column_data)  # リストからTensorFlowデータセットを作成します。
    dataset = dataset.batch(8)  # バッチサイズを8に設定します。

    embeddings = []  # 埋め込みを格納するリストを初期化します。
    for batch in dataset:  # データセットの各バッチに対してループします。
        batch_list = [str(text) for text in batch.numpy().tolist()]  # バッチ内のテキストをリストに変換します。
        batch_embeddings = get_bert_embeddings(batch_list)  # BERT埋め込みを取得します。
        embeddings.append(batch_embeddings)  # 埋め込みをリストに追加します。
    
    return np.concatenate(embeddings, axis=0)  # 埋め込みを結合し、1つのNumPy配列として返します。

# この関数は、与えられた列のテキストデータを処理し、BERTを使用して埋め込みを生成します。
# バッチ処理を使用することで、メモリ使用量を最適化し、GPUを効率的に活用できます。

In [ ]:
def add_embeddings_to_dataframe(df, column_names):  # データフレームに埋め込みを追加する関数を定義します。
    for column in column_names:  # 各指定された列に対してループします。
        print(f"Processing column: {column}")  # 現在処理中の列を表示します。
        embeddings = process_column(df[column])  # 指定された列に対して埋め込みを生成します。
        df[f'{column}_embedding'] = list(embeddings)  # 生成された埋め込みを新しい列としてデータフレームに追加します。
    return df  # 更新されたデータフレームを返します。

# この関数は、指定された列の埋め込みを計算し、それらを元のデータフレームに追加します。
# これにより、元のデータとその埋め込み表現を一緒に保持することができます。

In [ ]:
columns_to_embed = ['prompt_clean', 'response_a_clean', 'response_b_clean']  # 埋め込みを計算する列のリストを定義します。

# このリストには、クリーンなプロンプトおよびレスポンスのテキストデータが含まれており、
# 後でこれらの列に対してBERT埋め込みを生成するために使用されます。

In [ ]:
train_data = add_embeddings_to_dataframe(train_data, columns_to_embed)  # 指定された列に対して埋め込みを追加し、更新されたデータフレームを保存します。

# add_embeddings_to_dataframe関数を呼び出すことで、トレーニングデータに埋め込みを計算し、
# 新たに生成された埋め込みがデータフレームに追加されます。これにより、元のテキストデータとその埋め込みが一緒に使用できる状態になります。

In [ ]:
train_data['similarity_prompt_response_a'] = train_data.apply(  # プロンプトとレスポンスAのコサイン類似度を計算して新しい列を作成します。
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),  # プロンプト埋め込みを1次元の配列に変形します。
                                np.array(x['response_a_clean_embedding']).reshape(1, -1))[0][0], axis=1)  # レスポンスA埋め込みを1次元の配列に変形し、コサイン類似度を計算します。

train_data['similarity_prompt_response_b'] = train_data.apply(  # プロンプトとレスポンスBのコサイン類似度を計算して新しい列を作成します。
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),  # プロンプト埋め込みを1次元の配列に変形します。
                                np.array(x['response_b_clean_embedding']).reshape(1, -1))[0][0], axis=1)  # レスポンスB埋め込みを1次元の配列に変形し、コサイン類似度を計算します。

# apply()メソッドを使用して、各行に対してプロンプト埋め込みとレスポンス埋め込みのコサイン類似度を計算し、
# 結果を新しい列に格納します。コサイン類似度は、二つのベクトルの角度を基にした類似度の指標であり、
# 値が1に近いほど二つのベクトルが類似していることを意味します。

### データの準備

In [ ]:
X = train_data[['word_count_prompt', 'word_count_response_a', 'word_count_response_b',  # プロンプトおよびレスポンスAとBの単語数
                'char_count_prompt', 'char_count_response_a', 'char_count_response_b',  # プロンプトおよびレスポンスの文字数
                'lexical_diversity_prompt', 'lexical_diversity_response_a', 'lexical_diversity_response_b',  # 語彙の多様性
                'syllable_count_prompt', 'syllable_count_response_a', 'syllable_count_response_b',  # 音節数
                'sentence_count_prompt', 'sentence_count_response_a', 'sentence_count_response_b',  # 文の数
                'flesch_reading_ease_prompt', 'flesch_reading_ease_response_a', 'flesch_reading_ease_response_b',  # Flesch Reading Easeスコア
                'similarity_prompt_response_a', 'similarity_prompt_response_b',  # コサイン類似度
                'sentiment_prompt', 'sentiment_response_a', 'sentiment_response_b']]  # 感情スコア

# このコードは、モデルの入力に使用する特徴量（X）のデータフレームを作成しています。
# 各列はテキスト特徴量、感情スコア、コサイン類似度を表しており、
# 学習に用いるデータとして後で利用されます。

In [ ]:
# 目標列を定義します。
train_data['winner'] = train_data.apply(lambda x: 0 if x['winner_model_a'] == 1 else (1 if x['winner_model_b'] == 1 else 2), axis=1)  # モデルAの勝利を0、モデルBの勝利を1、引き分けを2として設定します。

# apply()メソッドを使用して、各行に対して条件に基づいて勝者のラベルを決定します。
# これにより、モデルの出力として使用される目標変数（winner）が作成されます。

In [ ]:
y = train_data['winner']  # 目標変数（勝者）をyとして定義します。

# このコードにより、トレーニングデータにおける各サンプルの勝者ラベルを含むシリーズが作成され、学習用モデルのターゲットとして使用されます。

### トレーニングデータとバリデーションデータの分割

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,  # データをトレーニングセットとバリデーションセットに分割します。
                                                  random_state=42)  # テストサイズを20%に設定し、乱数シードを42にします。

# train_test_split()関数は、指定されたデータ（特徴量Xとターゲットy）をトレーニングデータとバリデーションデータに分割します。
# random_stateを設定することで、分割結果を再現可能にします。これにより、毎回同じデータが分割されることが保証されます。

## モデルの定義

* ランダムフォレスト
* ロジスティック回帰
* サポートベクターマシン
* 勾配ブースティング
* ニューラルネットワーク

In [ ]:
models = {  # モデルの辞書を定義します。
    'Random Forest': RandomForestClassifier(),  # ランダムフォレスト分類器を追加します。
    'SVM': SVC(probability=True),  # サポートベクターマシンを追加し、確率予測を有効にします。
    'Gradient Boosting': GradientBoostingClassifier()  # 勾配ブースティング分類器を追加します。
}

# この辞書は、後で使用するための異なるモデルを格納し、
# モデルのトレーニングや評価を簡単に行えるようにします。

In [ ]:
# ニューラルネットワークを作成します。
def create_nn_model(input_shape):  # 入力形状を受け取る関数を定義します。
    model = Sequential()  # シーケンシャルモデルを初期化します。
    model.add(Dense(128, input_shape=(input_shape,), activation='relu'))  # 入力層と128のユニットを持つ全結合層を追加します。
    model.add(Dropout(0.2))  # ドロップアウト層を追加し、過学習を防ぎます。
    model.add(Dense(64, activation='relu'))  # 64のユニットを持つ全結合層を追加します。
    model.add(Dropout(0.2))  # 再度ドロップアウト層を追加します。
    model.add(Dense(3, activation='softmax'))  # 出力層を追加し、3つのクラスの確率を出力します。
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # モデルをコンパイルします。
    return model  # 定義したモデルを返します。

# この関数は、指定された入力形状を持つニューラルネットワークモデルを構築し、
# トレーニングに使用するための準備を行います。

In [ ]:
nn_model = create_nn_model(X_train.shape[1])  # トレーニングデータの特徴量の数に基づいてニューラルネットワークモデルを作成します。
models['Neural Network'] = nn_model  # 作成したニューラルネットワークモデルをモデルの辞書に追加します。

# これにより、ニューラルネットワークモデルが定義されたモデルのリストに追加され、
# 他のモデルと同様にトレーニングや評価が可能になります。

モデルのトレーニングと評価:

In [ ]:
results = {}  # モデルの評価結果を格納する辞書を初期化します。

for name, model in models.items():  # 定義されたすべてのモデルに対してループします。
    print(f"トレーニングと評価を行っています: {name}...")
    
    if name == 'Neural Network':  # ニューラルネットワークの場合
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=2)  # モデルをフィットし、検証データを使用します。
        y_pred = np.argmax(model.predict(X_val), axis=1)  # バリデーションデータに対する予測結果を取得します。
        y_pred_proba = model.predict(X_val)  # 予測確率を取得します。
    else:  # その他のモデルの場合
        model.fit(X_train, y_train)  # モデルをフィットします。
        y_pred = model.predict(X_val)  # バリデーションデータに対する予測結果を取得します。
        y_pred_proba = model.predict_proba(X_val)  # 予測確率を取得します。

    accuracy = accuracy_score(y_val, y_pred)  # 精度を計算します。
    logloss = log_loss(y_val, y_pred_proba)  # ログ損失を計算します。

    results[name] = {  # 結果を辞書に格納します。
        '精度': accuracy,
        'ログ損失': logloss,
        '分類レポート': classification_report(y_val, y_pred),
        '混同行列': confusion_matrix(y_val, y_pred)
    }

    print(f"{name}の精度: {accuracy}")  # モデルの精度を表示します。
    print(f"{name}のログ損失: {logloss}")  # モデルのログ損失を表示します。
    print(f"{name}の分類レポート:\n{classification_report(y_val, y_pred)}")  # 分類レポートを表示します。
    print(f"{name}の混同行列:\n{confusion_matrix(y_val, y_pred)}\n")  # 混同行列を表示します。

ログ損失に基づいて最良のモデルを選択しています。

In [ ]:
best_model_name = max(results, key=lambda name: results[name]['Log Loss'])  # 最小のログ損失を持つモデル名を取得します。
best_model = models[best_model_name]  # 最良のモデルを辞書から取得します。

# これにより、ログ損失が最も小さいモデルを選択し、
# そのモデルを後で使用するための変数best_modelに格納します。

## 予測と提出

テストデータの準備、テストデータに対する予測、および提出用データの作成を行います。

In [ ]:
# テストデータのテキストをクリーンアップします。
test_data['prompt_clean'] = test_data['prompt'].apply(clean_text)  # 'prompt'列にclean_text関数を適用し、クリーンアップした結果を'prompt_clean'列に格納します。
test_data['response_a_clean'] = test_data['response_a'].apply(clean_text)  # 'response_a'列にclean_text関数を適用し、クリーンアップした結果を'response_a_clean'列に格納します。
test_data['response_b_clean'] = test_data['response_b'].apply(clean_text)  # 'response_b'列にclean_text関数を適用し、クリーンアップした結果を'response_b_clean'列に格納します。

# このコードにより、テストデータのテキストが前処理され、モデルへの入力に適した形式になります。

In [ ]:
# テストデータのテキストをトークン化します。
test_sequences = tokenizer.texts_to_sequences(test_data['prompt_clean'])  # クリーンなプロンプトを数値のシーケンスに変換します。
response_a_test_sequences = tokenizer.texts_to_sequences(test_data['response_a_clean'])  # クリーンなレスポンスAを数値のシーケンスに変換します。
response_b_test_sequences = tokenizer.texts_to_sequences(test_data['response_b_clean'])  # クリーンなレスポンスBを数値のシーケンスに変換します。

# texts_to_sequences()メソッドを使用することで、テキストをトークンに変換し、
# モデルの入力として使用できる形式に準備します。

In [ ]:
# テストデータのシーケンスにパディングを行います。
test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post')  # プロンプトのシーケンスを最大長にパディングします。
response_a_test_sequences = pad_sequences(response_a_test_sequences, maxlen=max_len, padding='post')  # レスポンスAのシーケンスを最大長にパディングします。
response_b_test_sequences = pad_sequences(response_b_test_sequences, maxlen=max_len, padding='post')  # レスポンスBのシーケンスを最大長にパディングします。

# pad_sequences()を使用することで、すべてのシーケンスを指定した最大長に整形し、
# 短いシーケンスにはゼロパディングを追加して、モデルに入力できる形式にします。

In [ ]:
# テストデータに対して感情分析を行います。
test_data['sentiment_prompt'] = test_data['prompt_clean'].apply(sentiment_analysis)  # クリーニングされたプロンプトに感情分析を適用し、結果を'sentiment_prompt'列に格納します。
test_data['sentiment_response_a'] = test_data['response_a_clean'].apply(sentiment_analysis)  # クリーニングされたレスポンスAに感情分析を適用し、結果を'sentiment_response_a'列に格納します。
test_data['sentiment_response_b'] = test_data['response_b_clean'].apply(sentiment_analysis)  # クリーニングされたレスポンスBに感情分析を適用し、結果を'sentiment_response_b'列に格納します。

# このコードにより、テストデータの各テキストの感情スコアが計算され、
# 後のモデルに対する入力として使用できるようになります。

In [ ]:
# テストデータからテキスト構造の特徴量を作成します。
test_data['word_count_prompt'] = test_data['prompt_clean'].apply(word_count)  # プロンプトの単語数を計算し、'word_count_prompt'列に格納します。
test_data['word_count_response_a'] = test_data['response_a_clean'].apply(word_count)  # レスポンスAの単語数を計算し、'word_count_response_a'列に格納します。
test_data['word_count_response_b'] = test_data['response_b_clean'].apply(word_count)  # レスポンスBの単語数を計算し、'word_count_response_b'列に格納します。
test_data['char_count_prompt'] = test_data['prompt_clean'].apply(char_count)  # プロンプトの文字数を計算し、'char_count_prompt'列に格納します。
test_data['char_count_response_a'] = test_data['response_a_clean'].apply(char_count)  # レスポンスAの文字数を計算し、'char_count_response_a'列に格納します。
test_data['char_count_response_b'] = test_data['response_b_clean'].apply(char_count)  # レスポンスBの文字数を計算し、'char_count_response_b'列に格納します。
test_data['lexical_diversity_prompt'] = test_data['prompt_clean'].apply(lexical_diversity)  # プロンプトの語彙の多様性を計算し、'lexical_diversity_prompt'列に格納します。
test_data['lexical_diversity_response_a'] = test_data['response_a_clean'].apply(lexical_diversity)  # レスポンスAの語彙の多様性を計算し、'lexical_diversity_response_a'列に格納します。
test_data['lexical_diversity_response_b'] = test_data['response_b_clean'].apply(lexical_diversity)  # レスポンスBの語彙の多様性を計算し、'lexical_diversity_response_b'列に格納します。
test_data['syllable_count_prompt'] = test_data['prompt_clean'].apply(syllable_count)  # プロンプトの音節数を計算し、'syllable_count_prompt'列に格納します。
test_data['syllable_count_response_a'] = test_data['response_a_clean'].apply(syllable_count)  # レスポンスAの音節数を計算し、'syllable_count_response_a'列に格納します。
test_data['syllable_count_response_b'] = test_data['response_b_clean'].apply(syllable_count)  # レスポンスBの音節数を計算し、'syllable_count_response_b'列に格納します。
test_data['sentence_count_prompt'] = test_data['prompt_clean'].apply(sentence_count)  # プロンプトの文の数を計算し、'sentence_count_prompt'列に格納します。
test_data['sentence_count_response_a'] = test_data['response_a_clean'].apply(sentence_count)  # レスポンスAの文の数を計算し、'sentence_count_response_a'列に格納します。
test_data['sentence_count_response_b'] = test_data['response_b_clean'].apply(sentence_count)  # レスポンスBの文の数を計算し、'sentence_count_response_b'列に格納します。
test_data['flesch_reading_ease_prompt'] = test_data['prompt_clean'].apply(flesch_reading_ease)  # プロンプトのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_prompt'列に格納します。
test_data['flesch_reading_ease_response_a'] = test_data['response_a_clean'].apply(flesch_reading_ease)  # レスポンスAのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_response_a'列に格納します。
test_data['flesch_reading_ease_response_b'] = test_data['response_b_clean'].apply(flesch_reading_ease)  # レスポンスBのFlesch Reading Easeスコアを計算し、'flesch_reading_ease_response_b'列に格納します。

In [ ]:
# テストデータに埋め込みを追加します。
test_data = add_embeddings_to_dataframe(test_data, columns_to_embed)  # 定義した列に対して埋め込みを生成し、テストデータフレームに追加します。

# このコードにより、テストデータの各テキストに対してBERT埋め込みが計算され、
# 他の特徴量と一緒に保持されるようになります。これによって、モデルに対する入力が整います。

In [ ]:
# テストデータのコサイン類似度を計算します。
test_data['similarity_prompt_response_a'] = test_data.apply(  # プロンプト埋め込みとレスポンスA埋め込みのコサイン類似度を計算して新しい列を作成します。
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),  # プロンプト埋め込みを1次元の配列に変形します。
                                np.array(x['response_a_clean_embedding']).reshape(1, -1))[0][0], axis=1)  # レスポンスA埋め込みを1次元の配列に変形し、コサイン類似度を計算します。

test_data['similarity_prompt_response_b'] = test_data.apply(  # プロンプト埋め込みとレスポンスB埋め込みのコサイン類似度を計算して新しい列を作成します。
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),  # プロンプト埋め込みを1次元の配列に変形します。
                                np.array(x['response_b_clean_embedding']).reshape(1, -1))[0][0], axis=1)  # レスポンスB埋め込みを1次元の配列に変形し、コサイン類似度を計算します。

# apply()メソッドを使用して、各行に対してプロンプト埋め込みとレスポンス埋め込みのコサイン類似度を計算し、
# 結果を新しい列に格納します。コサイン類似度は、二つのベクトルの角度を基にした類似度の指標であり、
# 値が1に近いほど二つのベクトルが類似していることを意味します。

In [ ]:
X_test = test_data[['word_count_prompt', 'word_count_response_a', 'word_count_response_b',  # プロンプトおよびレスポンスAとBの単語数
                    'char_count_prompt', 'char_count_response_a', 'char_count_response_b',  # プロンプトおよびレスポンスの文字数
                    'lexical_diversity_prompt', 'lexical_diversity_response_a', 'lexical_diversity_response_b',  # 語彙の多様性
                    'syllable_count_prompt', 'syllable_count_response_a', 'syllable_count_response_b',  # 音節数
                    'sentence_count_prompt', 'sentence_count_response_a', 'sentence_count_response_b',  # 文の数
                    'flesch_reading_ease_prompt', 'flesch_reading_ease_response_a', 'flesch_reading_ease_response_b',  # Flesch Reading Easeスコア
                    'similarity_prompt_response_a', 'similarity_prompt_response_b',  # コサイン類似度
                    'sentiment_prompt', 'sentiment_response_a', 'sentiment_response_b']]  # 感情スコア

# このコードは、テストデータの特徴量（X_test）のデータフレームを作成しています。
# 各列はテキスト特徴量、感情スコア、コサイン類似度を表しており、
# モデルによる予測のために使用されます。

In [ ]:
test_pred_proba = best_model.predict(X_test)  # 最良のモデルを使用してテストデータに対する予測確率を計算します。

# このコードにより、テストデータに対して最良のモデルが予測を行い、
# 各クラスに対する確率が'test_pred_proba'に格納されます。これにより、後でクラスラベルを決定するために使用できます。

In [ ]:
submission = pd.DataFrame(test_data['id'])  # テストデータの'id'列を含む新しいデータフレームを作成します。
submission['winner_model_a'] = test_pred_proba[:, 0]  # モデルAの勝率を追加します。
submission['winner_model_b'] = test_pred_proba[:, 1]  # モデルBの勝率を追加します。
submission['winner_tie'] = test_pred_proba[:, 2]  # 引き分けの勝率を追加します。

submission.to_csv('submission.csv', index=False)  # 提出用データフレームをCSVファイルに保存します。

# このコードにより、予測結果を含む提出ファイルが作成され、コンペティションへの提出に利用できます。

In [ ]:
submission  # 作成した提出用データフレームを表示します。

# これにより、提出ファイルに含まれる内容を確認できます。各行にはテストデータのIDと、その勝者モデルに対する予測確率が含まれています。